In [1]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
from collections import Counter
import json
import time
import re

# 환경변수 로드
load_dotenv()

class BlockBasedTranslator:
    def __init__(self, target_language="ko"):
        # Azure 클라이언트 초기화
        self.vision_client = ImageAnalysisClient(
            endpoint=os.getenv("AZURE_VISION_ENDPOINT"),
            credential=AzureKeyCredential(os.getenv("AZURE_VISION_KEY"))
        )
        
        self.translator = TextTranslationClient(
            credential=AzureKeyCredential(os.getenv("TRANSLATOR_API_KEY")),
            endpoint=os.getenv("TRANSLATOR_ENDPOINT"),
            region=os.getenv("TRANSLATOR_REGION")
        )
        
        self.target_language = target_language
        
        # 번역 캐시
        self.translation_cache = {}
        self.load_translation_cache()
        
        print(f"✅ 블록 기반 번역기 초기화 완료 (구글 번역기 스타일)")
    
    def extract_text_blocks(self, image_path):
        """OCR로 텍스트 블록별 추출"""
        try:
            with open(image_path, "rb") as f:
                image_data = f.read()
            
            result = self.vision_client.analyze(
                image_data=image_data,
                visual_features=[VisualFeatures.READ]
            )
            
            text_blocks = []
            
            if result.read:
                for block_idx, block in enumerate(result.read.blocks):
                    # 블록 내 모든 텍스트 수집
                    block_texts = []
                    block_lines = []
                    
                    for line in block.lines:
                        text = line.text.strip()
                        if text:
                            block_texts.append(text)
                            
                            bbox = line.bounding_polygon
                            x_coords = [point.x for point in bbox]
                            y_coords = [point.y for point in bbox]
                            
                            x = min(x_coords)
                            y = min(y_coords)
                            width = max(x_coords) - x
                            height = max(y_coords) - y
                            
                            block_lines.append({
                                'text': text,
                                'bbox': (int(x), int(y), int(width), int(height))
                            })
                    
                    if block_texts:
                        # 블록 전체 바운딩 박스 계산
                        min_x = min(line['bbox'][0] for line in block_lines)
                        min_y = min(line['bbox'][1] for line in block_lines)
                        max_x = max(line['bbox'][0] + line['bbox'][2] for line in block_lines)
                        max_y = max(line['bbox'][1] + line['bbox'][3] for line in block_lines)
                        
                        # 블록 텍스트 병합
                        block_text = ' '.join(block_texts)
                        
                        text_blocks.append({
                            'block_id': block_idx,
                            'text': block_text,
                            'lines': block_lines,
                            'block_bbox': (min_x, min_y, max_x - min_x, max_y - min_y),
                            'line_count': len(block_lines)
                        })
            
            print(f"📦 추출된 텍스트 블록: {len(text_blocks)}개")
            for i, block in enumerate(text_blocks):
                print(f"   블록 {i+1}: '{block['text'][:80]}...' ({block['line_count']}줄)")
            
            return text_blocks
            
        except Exception as e:
            print(f"❌ OCR 오류: {e}")
            return []
    
    def smart_translate(self, text):
        """블록별 번역"""
        cache_key = f"{text}_{self.target_language}"
        if cache_key in self.translation_cache:
            return self.translation_cache[cache_key], "cached"
        
        try:
            response = self.translator.translate(
                body=[{"text": text}],
                to_language=[self.target_language]
            )
            
            if response and len(response) > 0:
                translated = response[0]['translations'][0]['text']
                detected_lang = response[0].get('detectedLanguage', {}).get('language', 'unknown')
                
                self.translation_cache[cache_key] = translated
                return translated, detected_lang
                
        except Exception as e:
            print(f"번역 오류: {e}")
        
        return text, "error"
    
    def split_translated_text_to_lines(self, translated_text, target_line_count):
        """번역된 텍스트를 목표 줄 수에 맞춰 분할"""
        if target_line_count == 1:
            return [translated_text]
        
        # 문장 단위로 먼저 분할 시도
        sentences = re.split(r'([.!?])', translated_text)
        # 구두점 재결합
        combined_sentences = []
        for i in range(0, len(sentences) - 1, 2):
            if i + 1 < len(sentences):
                combined_sentences.append(sentences[i] + sentences[i + 1])
            else:
                combined_sentences.append(sentences[i])
        
        if len(combined_sentences) >= target_line_count:
            # 문장을 줄 수에 맞춰 그룹화
            lines = []
            sentences_per_line = len(combined_sentences) // target_line_count
            remainder = len(combined_sentences) % target_line_count
            
            sent_idx = 0
            for i in range(target_line_count):
                num_sentences = sentences_per_line + (1 if i < remainder else 0)
                line_sentences = combined_sentences[sent_idx:sent_idx + num_sentences]
                lines.append(' '.join(line_sentences).strip())
                sent_idx += num_sentences
            
            return lines
        else:
            # 단어 단위로 분할
            words = translated_text.split()
            if len(words) <= target_line_count:
                # 단어가 적으면 각 줄에 균등 분배
                lines = []
                for i in range(target_line_count):
                    if i < len(words):
                        lines.append(words[i])
                    else:
                        lines.append("")
                return lines
            
            # 단어를 줄 수에 맞춰 분할
            words_per_line = len(words) // target_line_count
            remainder = len(words) % target_line_count
            
            lines = []
            word_idx = 0
            
            for i in range(target_line_count):
                num_words = words_per_line + (1 if i < remainder else 0)
                num_words = max(1, num_words)
                
                line_words = words[word_idx:word_idx + num_words]
                lines.append(' '.join(line_words))
                word_idx += num_words
            
            return lines
    
    def get_korean_font(self, font_size):
        """한글 폰트 가져오기"""
        try:
            font_paths = [
                "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
                "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf",
                "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
                "/System/Library/Fonts/AppleGothic.ttf",  # macOS
                "C:\\Windows\\Fonts\\malgun.ttf"  # Windows
            ]
            
            for font_path in font_paths:
                try:
                    return ImageFont.truetype(font_path, size=font_size)
                except:
                    continue
            
            return ImageFont.load_default()
            
        except Exception:
            return ImageFont.load_default()
    
    def create_block_overlay(self, image_pil, draw, block, translated_lines):
        """개별 블록에 대한 오버레이 생성"""
        if not translated_lines or not block['lines']:
            return image_pil
        
        print(f"      📝 블록 오버레이 생성: {len(translated_lines)}줄")
        
        for i, (original_line, translated_line) in enumerate(zip(block['lines'], translated_lines)):
            if not translated_line or not translated_line.strip():
                continue
            
            # 원본 줄 정보
            orig_x, orig_y, orig_w, orig_h = original_line['bbox']
            
            # 폰트 크기 설정 (원본 줄 높이 기준)
            font_size = max(min(int(orig_h * 0.85), 40), 14)
            font = self.get_korean_font(font_size)
            
            # 텍스트 크기 측정
            try:
                bbox_coords = draw.textbbox((0, 0), translated_line, font=font)
                text_width = bbox_coords[2] - bbox_coords[0]
                text_height = bbox_coords[3] - bbox_coords[1]
            except:
                text_width, text_height = draw.textsize(translated_line, font=font)
            
            # 텍스트가 원본 영역보다 너무 길면 폰트 크기 조정
            if text_width > orig_w * 1.5:  # 원본의 1.5배를 초과하면
                scale_factor = (orig_w * 1.5) / text_width
                font_size = max(int(font_size * scale_factor), 10)
                font = self.get_korean_font(font_size)
                
                # 다시 측정
                try:
                    bbox_coords = draw.textbbox((0, 0), translated_line, font=font)
                    text_width = bbox_coords[2] - bbox_coords[0]
                    text_height = bbox_coords[3] - bbox_coords[1]
                except:
                    text_width, text_height = draw.textsize(translated_line, font=font)
            
            # 배경 박스 설정
            padding = 6
            box_width = text_width + padding * 2
            box_height = text_height + padding * 2
            
            # 박스 위치 (원본 줄 위치 기준)
            box_x = orig_x
            box_y = orig_y + (orig_h - box_height) // 2
            
            # 이미지 경계 조정
            box_x = min(max(0, box_x), image_pil.width - box_width)
            box_y = min(max(0, box_y), image_pil.height - box_height)
            
            # 반투명 배경 생성
            overlay = Image.new("RGBA", image_pil.size, (255, 255, 255, 0))
            overlay_draw = ImageDraw.Draw(overlay)
            
            # 배경색 (구글 번역기 스타일)
            bg_color = (255, 255, 255, 200)  # 불투명한 흰색
            overlay_draw.rectangle([box_x, box_y, box_x + box_width, box_y + box_height], 
                                 fill=bg_color, outline=(200, 200, 200, 255), width=1)
            
            # 합성
            image_pil = Image.alpha_composite(image_pil.convert("RGBA"), overlay).convert("RGB")
            draw = ImageDraw.Draw(image_pil)
            
            # 텍스트 그리기
            text_x = box_x + padding
            text_y = box_y + padding
            
            draw.text((text_x, text_y), translated_line, font=font, fill=(0, 0, 0))
            
            print(f"        줄 {i+1}: '{original_line['text'][:30]}...' → '{translated_line[:30]}...'")
        
        return image_pil
    
    def create_google_style_overlay(self, image_path, text_blocks, translations):
        """구글 번역기 스타일 오버레이 생성"""
        image = cv2.imread(image_path)
        image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(image_pil)
        
        print(f"🎨 구글 스타일 오버레이 생성: {len(text_blocks)}개 블록")
        
        for block_idx, (block, translation_info) in enumerate(zip(text_blocks, translations)):
            translated_text = translation_info['translated']
            
            print(f"   📦 블록 {block_idx + 1} 처리:")
            print(f"      원본: '{translation_info['original'][:60]}...'")
            print(f"      번역: '{translated_text[:60]}...'")
            
            if not translated_text.strip():
                continue
            
            # 번역된 텍스트를 원본 줄 수에 맞춰 분할
            target_line_count = block['line_count']
            translated_lines = self.split_translated_text_to_lines(translated_text, target_line_count)
            
            print(f"      🔄 {len(translated_lines)}줄로 분할")
            
            # 블록별 오버레이 생성
            image_pil = self.create_block_overlay(image_pil, draw, block, translated_lines)
            draw = ImageDraw.Draw(image_pil)  # draw 객체 갱신
        
        return cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
    
    def process_image(self, image_path):
        """구글 번역기 스타일 이미지 처리"""
        print(f"🔍 구글 스타일 블록 번역 시작: {image_path}")
        
        # 1. 텍스트 블록별 추출
        text_blocks = self.extract_text_blocks(image_path)
        
        if not text_blocks:
            print("❌ 추출된 텍스트 블록이 없습니다.")
            return None
        
        # 2. 각 블록별로 개별 번역
        translations = []
        
        for block_idx, block in enumerate(text_blocks):
            print(f"📝 블록 {block_idx + 1} 번역 중...")
            
            block_text = block['text']
            translated_text, status = self.smart_translate(block_text)
            
            translations.append({
                'original': block_text,
                'translated': translated_text,
                'status': status
            })
            
            print(f"   ✅ 완료 ({status})")
        
        # 3. 구글 스타일 오버레이 생성
        result_image = self.create_google_style_overlay(image_path, text_blocks, translations)
        
        # 4. 저장
        output_path = f"google_style_translated_{os.path.basename(image_path)}"
        cv2.imwrite(output_path, result_image)
        
        print(f"💾 결과 저장: {output_path}")
        
        # 통계
        status_counts = Counter([t['status'] for t in translations])
        print(f"📊 번역 통계: {dict(status_counts)}")
        print(f"✨ 구글 스타일 블록 번역 완료!")
        
        return result_image, output_path
    
    def load_translation_cache(self):
        """번역 캐시 로드"""
        try:
            if os.path.exists("translation_cache.json"):
                with open("translation_cache.json", "r", encoding="utf-8") as f:
                    self.translation_cache = json.load(f)
                print(f"📋 번역 캐시 로드: {len(self.translation_cache)}개 항목")
        except Exception as e:
            print(f"캐시 로드 실패: {e}")
            self.translation_cache = {}
    
    def save_translation_cache(self):
        """번역 캐시 저장"""
        try:
            with open("translation_cache.json", "w", encoding="utf-8") as f:
                json.dump(self.translation_cache, f, ensure_ascii=False, indent=2)
            print(f"💾 번역 캐시 저장: {len(self.translation_cache)}개 항목")
        except Exception as e:
            print(f"캐시 저장 실패: {e}")

def main():
    print("🔍 구글 번역기 스타일 블록 번역기")
    print("📦 각 텍스트 블록을 정확한 위치에 개별 번역")
    print("=" * 60)
    
    translator = BlockBasedTranslator()
    
    while True:
        print(f"\n현재 타겟 언어: 한국어")
        print("=" * 40)
        print("1. 구글 스타일 블록 번역")
        print("2. 번역 캐시 현황")
        print("0. 종료")
        print("=" * 40)
        
        choice = input("선택하세요 > ").strip()
        
        if choice == "1":
            image_path = input("이미지 파일 경로 입력 > ").strip()
            if os.path.exists(image_path):
                print("\n🔄 처리 중...")
                result = translator.process_image(image_path)
                
                if result is not None:
                    result_image, output_path = result
                    print("✅ 구글 스타일 번역 완료!")
                    
                    cv2.imshow('Google Style Block Translator', result_image)
                    print(f"📱 결과 저장: {output_path}")
                    print("⏳ 아무 키나 누르면 계속...")
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                else:
                    print("ℹ️ 처리할 텍스트가 없습니다.")
            else:
                print("❌ 파일이 존재하지 않습니다.")
                
        elif choice == "2":
            print(f"📋 번역 캐시: {len(translator.translation_cache)}개 항목 저장됨")
            
        elif choice == "0":
            translator.save_translation_cache()
            print("👋 프로그램을 종료합니다.")
            break
            
        else:
            print("❌ 잘못된 선택입니다.")

if __name__ == "__main__":
    main()


🔍 구글 번역기 스타일 블록 번역기
📦 각 텍스트 블록을 정확한 위치에 개별 번역
✅ 블록 기반 번역기 초기화 완료 (구글 번역기 스타일)

현재 타겟 언어: 한국어
1. 구글 스타일 블록 번역
2. 번역 캐시 현황
0. 종료


KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import json
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import time
from typing import List, Dict, Tuple
from dotenv import load_dotenv

# 최신 Azure AI Vision SDK 사용
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient

# 환경변수 로드
load_dotenv()

class AzureOCRTranslatorSDK:
    def __init__(self, target_language: str = 'ko'):
        """
        Azure AI Vision SDK와 Translator SDK를 사용하는 클래스
        """
        # 환경변수에서 설정값 로드 및 확인
        self.vision_endpoint = os.getenv("AZURE_VISION_ENDPOINT")
        self.vision_key = os.getenv("AZURE_VISION_KEY")
        self.translator_endpoint = os.getenv("TRANSLATOR_ENDPOINT")
        self.translator_key = os.getenv("TRANSLATOR_API_KEY")
        self.translator_region = os.getenv("TRANSLATOR_REGION")
        self.target_language = target_language
        
        required_vars = {
            'AZURE_VISION_ENDPOINT': self.vision_endpoint, 'AZURE_VISION_KEY': self.vision_key,
            'TRANSLATOR_ENDPOINT': self.translator_endpoint, 'TRANSLATOR_API_KEY': self.translator_key,
            'TRANSLATOR_REGION': self.translator_region
        }
        missing_vars = [var for var, value in required_vars.items() if not value]
        if missing_vars:
            raise ValueError(f"다음 환경변수가 .env 파일에 정의되지 않았습니다: {', '.join(missing_vars)}")
        
        # Azure 클라이언트 초기화
        try:
            self.vision_client = ImageAnalysisClient(
                endpoint=self.vision_endpoint,
                credential=AzureKeyCredential(self.vision_key)
            )
            self.translator_client = TextTranslationClient(
                endpoint=self.translator_endpoint,
                credential=AzureKeyCredential(self.translator_key),
                region=self.translator_region
            )
        except Exception as e:
            raise ValueError(f"Azure 클라이언트 초기화 실패: {e}")
        
        print(f"✅ Azure OCR 번역기 SDK 초기화 완료 (기본 언어: {target_language})")

    def extract_text_from_image(self, image_path: str) -> List[Dict]:
        """
        이미지에서 텍스트를 블록과 라인 단위로 추출
        """
        with open(image_path, "rb") as f:
            image_data = f.read()
        
        result = self.vision_client.analyze(image_data=image_data, visual_features=[VisualFeatures.READ])
        
        text_blocks = []
        if result.read and result.read.blocks:
            for block in result.read.blocks:
                all_points_x = [p.x for line in block.lines for p in line.bounding_polygon]
                all_points_y = [p.y for line in block.lines for p in line.bounding_polygon]
                if not all_points_x: continue

                min_x, max_x = min(all_points_x), max(all_points_x)
                min_y, max_y = min(all_points_y), max(all_points_y)
                
                text_blocks.append({
                    'text': " ".join([line.text for line in block.lines]),
                    'bbox': {'x': min_x, 'y': min_y, 'width': max_x - min_x, 'height': max_y - min_y},
                    'lines': [{'text': line.text, 'bbox_polygon': line.bounding_polygon} for line in block.lines]
                })
        return text_blocks

    def translate_text(self, text: str, target_language: str = None) -> str:
        """텍스트 번역 함수"""
        if not text.strip(): return ""
        lang = target_language or self.target_language
        
        try:
            response = self.translator_client.translate(body=[{"text": text}], to_language=[lang])
            return response[0]['translations'][0]['text'] if response else text
        except Exception as e:
            print(f"번역 오류 발생: {e}")
            raise e # 오류를 다시 발생시켜 상위 except 블록에서 처리하도록 함

    def get_font(self, size: int = 16):
        """시스템에 맞는 한글 폰트 반환"""
        font_paths = [
            "C:\\Windows\\Fonts\\malgunbd.ttf", "/System/Library/Fonts/AppleSDGothicNeoB.ttc",
            "/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf"
        ]
        for font_path in font_paths:
            if os.path.exists(font_path):
                try: return ImageFont.truetype(font_path, size)
                except IOError: continue
        return ImageFont.load_default()

    # --- 개선 사항: 번역 상태(status)를 인자로 받아 실패 시 특정 색상 반환 ---
    def get_dynamic_colors(self, image_np: np.ndarray, bbox: Dict, status: str) -> Tuple[Tuple, Tuple]:
        """bbox 영역 배경색 분석 및 상태에 따라 텍스트/배경색 결정"""
        
        # 번역 실패 시, 가독성보다 '실패'를 알리는 데 집중
        if status == 'failed':
            return (255, 255, 255), (220, 53, 69, 190) # 흰색 글씨, 붉은색 반투명 배경
        
        x, y, w, h = int(bbox['x']), int(bbox['y']), int(bbox['width']), int(bbox['height'])
        x1, y1 = max(0, x), max(0, y)
        x2, y2 = min(image_np.shape[1], x + w), min(image_np.shape[0], y + h)
        
        if x1 >= x2 or y1 >= y2: return (255, 255, 255), (0, 0, 0, 180)

        region = image_np[y1:y2, x1:x2]
        if region.size == 0: return (255, 255, 255), (0, 0, 0, 180)

        avg_color = np.mean(region, axis=(0, 1))
        luminance = (0.299 * avg_color[2] + 0.587 * avg_color[1] + 0.114 * avg_color[0])
        
        if luminance > 128: # 배경이 밝으면
            return (30, 30, 30), (255, 255, 255, 180)
        else: # 배경이 어두우면
            return (255, 255, 255), (0, 0, 0, 180)
    
    def group_text_lines(self, text_blocks: List[Dict], max_distance_ratio: float = 1.0) -> List[List[Dict]]:
        """텍스트 블록을 문단 단위로 그룹화"""
        if not text_blocks: return []
        sorted_blocks = sorted(text_blocks, key=lambda x: x['bbox']['y'])
        avg_height = np.mean([block['bbox']['height'] for block in sorted_blocks])
        max_distance = avg_height * max_distance_ratio

        groups, current_group = [], [sorted_blocks[0]]
        for block in sorted_blocks[1:]:
            prev_bottom_y = current_group[-1]['bbox']['y'] + current_group[-1]['bbox']['height']
            if block['bbox']['y'] - prev_bottom_y <= max_distance:
                current_group.append(block)
            else:
                groups.append(current_group)
                current_group = [block]
        groups.append(current_group)
        return groups

    def split_translation_to_lines(self, translation: str, original_lines: List[str]) -> List[str]:
        """번역된 텍스트를 원본 줄 수에 맞춰 분할"""
        if len(original_lines) <= 1: return [translation]
        
        words = translation.split()
        if not words: return [""] * len(original_lines)
        
        # 원본 줄의 글자 수 비율에 따라 단어 배분
        original_lengths = [len(line) for line in original_lines]
        total_len = sum(original_lengths)
        if total_len == 0: return [translation] + [""] * (len(original_lines) -1)

        words_per_line = [int(len(words) * (l / total_len)) for l in original_lengths]
        
        # 남는 단어 배분
        rem_words = len(words) - sum(words_per_line)
        for i in range(rem_words): words_per_line[i % len(words_per_line)] += 1
        
        result_lines, word_idx = [], 0
        for count in words_per_line:
            result_lines.append(" ".join(words[word_idx : word_idx + count]))
            word_idx += count
        
        return result_lines

    def create_overlay_image(self, image_path: str, all_text_blocks: List[Dict], translated_data: List[Dict], output_path: str = None) -> str:
        """번역 결과를 이미지에 오버레이"""
        image_np_bgr = cv2.imread(image_path)
        pil_image = Image.fromarray(cv2.cvtColor(image_np_bgr, cv2.COLOR_BGR2RGB)).convert("RGBA")
        overlay = Image.new('RGBA', pil_image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(overlay)

        for data in translated_data:
            block = all_text_blocks[data['original_block_idx']]
            status = data['status']
            
            for line_idx, (original_line, translated_line) in enumerate(zip(block['lines'], data['translated_lines'])):
                if not translated_line.strip(): continue # 빈 줄은 그리지 않음

                poly = original_line['bbox_polygon']
                x_coords = [p.x for p in poly]
                y_coords = [p.y for p in poly]
                line_bbox = {'x': min(x_coords), 'y': min(y_coords), 'width': max(x_coords) - min(x_coords), 'height': max(y_coords) - min(y_coords)}

                text_color, bg_color = self.get_dynamic_colors(image_np_bgr, line_bbox, status)
                
                font_size = max(min(int(line_bbox['height'] * 0.7), 40), 10)
                font = self.get_font(font_size)
                
                try: text_width, text_height = font.getlength(translated_line), font.size
                except AttributeError: text_width, text_height = font.getsize(translated_line)
                
                padding, radius = 5, 8
                bg_x1, bg_y1 = int(line_bbox['x']) - padding, int(line_bbox['y']) - padding
                bg_x2 = int(line_bbox['x'] + max(line_bbox['width'], text_width)) + padding
                bg_y2 = int(line_bbox['y'] + max(line_bbox['height'], text_height)) + padding

                draw.rounded_rectangle([(bg_x1, bg_y1), (bg_x2, bg_y2)], radius=radius, fill=bg_color)
                
                text_x = bg_x1 + (bg_x2 - bg_x1 - text_width) / 2
                text_y = bg_y1 + (bg_y2 - bg_y1 - text_height) / 2
                
                draw.text((text_x, text_y), translated_line, fill=text_color, font=font)
                print(f"🎨 [{status}] '{original_line['text']}' -> '{translated_line}'")
        
        final_image = Image.alpha_composite(pil_image, overlay).convert('RGB')
        output_path = output_path or f"{os.path.splitext(os.path.basename(image_path))[0]}_translated.png"
        final_image.save(output_path)
        return output_path
    
    def process_image(self, image_path: str, target_language: str = None, output_path: str = None, group_lines: bool = True) -> str:
        print(f"\n🖼️ 이미지 처리 시작: {image_path}")
        all_text_blocks = self.extract_text_from_image(image_path)
        if not all_text_blocks:
            print("❌ 추출된 텍스트가 없습니다."); return None
        
        print(f"📦 추출된 텍스트 블록 수: {len(all_text_blocks)}")
        translated_data_for_overlay = []

        if group_lines:
            text_groups = self.group_text_lines(all_text_blocks)
            print(f"📚 {len(text_groups)}개 문단으로 그룹화됨")

            for group in text_groups:
                combined_text = ' '.join([block['text'] for block in group])
                original_lines_in_group = [line['text'] for block in group for line in block['lines']]
                
                try:
                    # 문단 번역 시도
                    combined_translation = self.translate_text(combined_text, target_language)
                    split_translations = self.split_translation_to_lines(combined_translation, original_lines_in_group)
                    status = 'success'
                    print(f"✅ 문단 번역 성공: '{combined_text}' -> '{combined_translation}'")
                
                # --- 개선 사항: 번역 실패 시 except 블록에서 원본 텍스트로 폴백 ---
                except Exception as e:
                    # 번역 실패 시 원본 텍스트를 그대로 사용
                    split_translations = original_lines_in_group 
                    status = 'failed'
                    print(f"❌ 문단 번역 실패: '{combined_text}'. 원본 텍스트로 대체합니다.")

                # 그룹 내 각 블록에 대해 번역 결과(또는 원본)를 매핑
                line_cursor = 0
                for block in group:
                    num_lines_in_block = len(block['lines'])
                    translated_data_for_overlay.append({
                        'original_block_idx': all_text_blocks.index(block),
                        'translated_lines': split_translations[line_cursor : line_cursor + num_lines_in_block],
                        'status': status
                    })
                    line_cursor += num_lines_in_block
        else: # 개별 블록 번역
            for idx, block in enumerate(all_text_blocks):
                try:
                    translated_text = self.translate_text(block['text'], target_language)
                    status = 'success'
                except Exception:
                    translated_text = block['text'] # 실패 시 원본
                    status = 'failed'
                
                translated_lines = self.split_translation_to_lines(translated_text, [line['text'] for line in block['lines']])
                translated_data_for_overlay.append({
                    'original_block_idx': idx,
                    'translated_lines': translated_lines,
                    'status': status
                })

        print("🎨 오버레이 이미지 생성 중...")
        result_path = self.create_overlay_image(image_path, all_text_blocks, translated_data_for_overlay, output_path)
        print(f"✅ 처리 완료! 결과 파일: {result_path}")
        return result_path

def main():
    """메인 실행 함수"""
    print("=" * 60)
    print("🌍 Azure OCR 번역기 (SDK v2.0 - 안정성 개선)")
    print("=" * 60)
    
    try:
        translator = AzureOCRTranslatorSDK(target_language='ko')
        while True:
            print("\n" + "=" * 40 + "\n1. 이미지 번역\n2. 종료\n" + "=" * 40)
            choice = input("선택하세요 > ").strip()
            
            if choice == "1":
                image_path = input("처리할 이미지 파일 경로를 입력하세요 > ").strip().replace("'", "").replace('"', '')
                if not os.path.exists(image_path):
                    print("❌ 파일이 존재하지 않습니다."); continue
                
                target_lang = input("번역할 언어 코드를 입력하세요 (기본값: ko) > ").strip() or "ko"
                group_option = input("문단 단위로 번역하시겠습니까? (y/n, 기본값: y) > ").strip().lower()
                
                result_path = translator.process_image(image_path, target_lang, group_lines=(group_option != 'n'))
                
                if result_path and input("\n결과 이미지를 표시하시겠습니까? (y/n) > ").strip().lower() == 'y':
                    try:
                        cv2.imshow('Translation Result', cv2.imread(result_path))
                        print("⏳ 아무 키나 눌러서 계속..."); cv2.waitKey(0); cv2.destroyAllWindows()
                    except Exception as e:
                        print(f"이미지 표시 실패: {e}. 파일은 {result_path}에 저장되었습니다.")
            
            elif choice == "2":
                print("👋 프로그램을 종료합니다."); break
            else:
                print("❌ 잘못된 선택입니다.")
                
    except Exception as e:
        print(f"❌ 치명적 오류 발생: {e}")
        # 필요한 패키지 안내 등 추가 가능

if __name__ == "__main__":
    main()

🌍 Azure OCR 번역기 (SDK v2.0 - 안정성 개선)
✅ Azure OCR 번역기 SDK 초기화 완료 (기본 언어: ko)

1. 이미지 번역
2. 종료

🖼️ 이미지 처리 시작: 222.jpg
📦 추출된 텍스트 블록 수: 1
📚 1개 문단으로 그룹화됨
✅ 문단 번역 성공: 'Claude Monet French, 1840-1926 Red Boats, Argenteuil, 1875 Oil on canvas Fogg Museum, Bequest from the Collection of Maurice Wertheim, Class of 1906, 1951,54 For most of the 1870s, Monet lived in Argenteuil, a town on the Seine only seventeen miles from Paris and accessible to the city by train. He often worked outdoors in his own boat and painted this basin several times, though the lower vantage point of this scene suggests a perspective from the shoreline. Eliminating the factory smokestacks that were in fact visible at this location, Monet instead highlights the area's picturesque and recreational aspects, including the boat rental area on the left. Impressionist painters like Monet deliberately eschewed the finished structure and line of academic painting, but the irresolution of this painting has led to its occasional cat

In [ ]:
import os
import cv2
import re
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient

# Load environment variables from .env file
load_dotenv()

class ProfessionalImageTranslator:
    def __init__(self, target_language='ko'):
        """Initializes the translator with Azure credentials."""
        try:
            self.vision_endpoint = os.getenv("AZURE_VISION_ENDPOINT")
            self.vision_key = os.getenv("AZURE_VISION_KEY")
            self.translator_endpoint = os.getenv("TRANSLATOR_ENDPOINT")
            self.translator_key = os.getenv("TRANSLATOR_API_KEY")
            self.translator_region = os.getenv("TRANSLATOR_REGION")
            self.target_language = target_language

            # Initialize Azure clients
            self.vision_client = ImageAnalysisClient(
                endpoint=self.vision_endpoint,
                credential=AzureKeyCredential(self.vision_key)
            )
            self.translator_client = TextTranslationClient(
                endpoint=self.translator_endpoint,
                credential=AzureKeyCredential(self.translator_key),
                region=self.translator_region
            )
            print("✅ 전문가급 이미지 번역기가 성공적으로 초기화되었습니다.")
        except Exception as e:
            print(f"❌ 초기화에 실패했습니다. .env 파일과 자격 증명을 확인하세요. 오류: {e}")
            raise

    def extract_text_blocks_grouped(self, image_path):
        """
        Extracts text from an image, treating each detected line as a separate block
        to preserve layouts like columns in menus.
        """
        with open(image_path, "rb") as f:
            image_data = f.read()

        print("👁️ 이미지를 분석하여 텍스트 위치를 파악하는 중...")
        result = self.vision_client.analyze(
            image_data=image_data,
            visual_features=[VisualFeatures.READ]
        )

        text_blocks = []
        if result.read and result.read.blocks:
            # Iterate through every single line detected, ignoring the default block grouping
            all_lines = [line for block in result.read.blocks for line in block.lines]
            
            for line in all_lines:
                points = line.bounding_polygon
                x_coords = [p.x for p in points]
                y_coords = [p.y for p in points]
                
                line_info = {
                    'text': line.text,
                    'x': min(x_coords),
                    'y': min(y_coords),
                    'width': max(x_coords) - min(x_coords),
                    'height': max(y_coords) - min(y_coords)
                }

                # Create a new "block" for each line to process it independently
                text_blocks.append({
                    'text': line.text,
                    'x': line_info['x'],
                    'y': line_info['y'],
                    'width': line_info['width'],
                    'height': line_info['height'],
                    'lines': [line_info],  # The block contains only this one line
                    'line_count': 1
                })

        print(f"📦 {len(text_blocks)}개의 개별 텍스트 라인을 블록으로 정리했습니다.")
        return text_blocks

    def translate_text(self, text):
        """Translates a string of text."""
        if not text.strip():
            return ""
        # The calling function will handle the exception
        response = self.translator_client.translate(
            body=[{"text": text}],
            to_language=[self.target_language]
        )
        if response and len(response) > 0:
            return response[0]['translations'][0]['text']
        return text

    def get_premium_font(self, size, weight='regular'):
        """Loads a high-quality local font, with fallbacks."""
        font_paths = {
            'bold': [
                "C:\\Windows\\Fonts\\malgunbd.ttf",  # Malgun Gothic Bold
                "/System/Library/Fonts/Supplemental/AppleGothic.ttf",
                "/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf"
            ],
            'regular': [
                "C:\\Windows\\Fonts\\malgun.ttf",  # Malgun Gothic Regular
                "/System/Library/Fonts/Supplemental/AppleGothic.ttf",
                "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"
            ]
        }
        for font_path in font_paths.get(weight, font_paths['regular']):
            if os.path.exists(font_path):
                try:
                    return ImageFont.truetype(font_path, size)
                except IOError:
                    continue
        return ImageFont.load_default()

    def split_translation_smartly(self, translation, original_lines):
        """Intelligently splits translated text to fit the original line count."""
        # Since each block now has only one line, this function is simplified.
        return [translation]

    def create_artistic_overlay(self, image_path, text_blocks, translations, output_path=None):
        """🎨 Creates a clean, artistic overlay by placing text in boxes line-by-line."""
        print("🎨 세련된 예술적 오버레이를 생성 중입니다...")
        
        pil_image = Image.open(image_path).convert("RGBA")
        overlay = Image.new('RGBA', pil_image.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(overlay)

        for block, translation in zip(text_blocks, translations):
            if not translation.strip():
                continue

            # Since each block is one line, we directly access the first (and only) line
            original_line = block['lines'][0]
            trans_line = translation

            # --- 1. Determine base font properties ---
            # Simplified logic as we don't have multi-line titles in this mode
            font_weight = 'regular' 
            font_size = max(int(original_line['height'] * 0.9), 10)
            font = self.get_premium_font(font_size, font_weight)
            
            # --- 2. Adapt font size to fit the available width ---
            max_width = original_line['width'] * 1.1 # Allow text to be slightly wider
            
            # Reduce font size until the text fits
            while font.getlength(trans_line) > max_width and font_size > 9:
                font_size -= 1
                font = self.get_premium_font(font_size, font_weight)

            # --- 3. Calculate final text dimensions ---
            try:
                bbox = draw.textbbox((0, 0), trans_line, font=font)
                text_width = bbox[2] - bbox[0]
                text_height = bbox[3] - bbox[1]
                text_y_offset = bbox[1]
            except AttributeError:
                text_width, text_height = draw.textsize(trans_line, font=font)
                text_y_offset = 0

            # --- 4. Define the background box for this line ---
            padding_x = int(font_size * 0.4)
            padding_y = int(font_size * 0.2)
            
            box_width = text_width + padding_x * 2
            box_height = text_height + padding_y * 2
            
            # Center the new, perfectly sized box over the original line's area
            box_x = original_line['x'] + (original_line['width'] - box_width) / 2
            box_y = original_line['y'] + (original_line['height'] - box_height) / 2

            # --- 5. Draw the semi-transparent background ---
            fill_color = (255, 230, 230, 210) # Light pink, ~82% opaque
            draw.rounded_rectangle(
                [(box_x, box_y), (box_x + box_width, box_y + box_height)],
                radius=6,
                fill=fill_color
            )
            
            # --- 6. Draw the text, centered within its new background ---
            text_x = box_x + padding_x
            text_y = box_y + padding_y - text_y_offset
            
            text_color = (30, 30, 30)
            draw.text(
                (text_x, text_y),
                trans_line,
                fill=text_color,
                font=font
            )
        
        print(f"  ✨ {len(text_blocks)}개의 라인을 성공적으로 렌더링했습니다.")

        # Composite the overlay onto the original image
        final_image = Image.alpha_composite(pil_image, overlay).convert('RGB')

        if output_path is None:
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            output_path = f"{base_name}_artistic_translated.png"

        final_image.save(output_path, "PNG", quality=95)
        print(f"💎 예술적 결과물이 {output_path}에 저장되었습니다.")
        return output_path

    def translate_image(self, image_path, target_language=None, output_path=None):
        """Main function to translate an image with an artistic overlay."""
        if target_language:
            self.target_language = target_language
        
        print(f"\n🎨 예술적 이미지 번역을 시작합니다: {image_path}")
        
        try:
            text_blocks = self.extract_text_blocks_grouped(image_path)
            if not text_blocks:
                print("❌ 이미지에서 텍스트를 찾을 수 없습니다.")
                return None
            
            print("🔄 텍스트 블록을 번역 중입니다...")
            translations = [self.translate_text(block['text']) for block in text_blocks]
            
            # Call the artistic overlay function
            result_path = self.create_artistic_overlay(
                image_path, text_blocks, translations, output_path
            )
            
            print(f"✨ 번역 완료! 결과: {result_path}")
            return result_path
            
        except Exception as e:
            print(f"❌ 번역 중 오류가 발생했습니다: {e}")
            # Improved error message for the user
            if "400036" in str(e) or "invalid" in str(e).lower():
                print("\n🤔 오류 메시지는 '언어 코드'를 언급하지만, Azure API 키, 엔드포인트 또는 지역이 잘못되었을 때도 자주 발생합니다.")
                print("👉 .env 파일의 TRANSLATOR_API_KEY, TRANSLATOR_ENDPOINT, TRANSLATOR_REGION 값이 올바른지 다시 한번 확인해주세요.")
            return None

def main():
    """Main loop to run the translator from the command line."""
    # List of common language codes to help the user
    supported_languages = {
        'ko': '한국어', 'en': '영어', 'ja': '일본어', 'zh-Hans': '중국어(간체)',
        'fr': '프랑스어', 'de': '독일어', 'es': '스페인어'
    }

    print("=" * 60)
    print("🎨 예술적 Azure 이미지 번역기")
    print("💎 아름다운 박물관 스타일 오버레이를 생성합니다.")
    print("=" * 60)
    
    try:
        translator = ProfessionalImageTranslator(target_language='ko')
        
        while True:
            print("\n" + "=" * 40)
            print("1. 이미지 번역하기")
            print("2. 종료")
            print("=" * 40)
            
            choice = input("옵션을 선택하세요 > ").strip()
            
            if choice == "1":
                image_path = input("이미지 파일 경로를 입력하세요 > ").strip().replace('"', '')
                
                if not os.path.exists(image_path):
                    print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
                    continue
                
                # Guide the user with language options
                print("\n📖 번역 가능한 언어 코드 예시:")
                for code, name in supported_languages.items():
                    print(f"  - {code}: {name}")

                while True:
                    target_lang = input("\n번역할 언어 코드를 입력하세요 (기본값: ko) > ").strip() or 'ko'
                    # A simple validation check (can be expanded)
                    if len(target_lang) > 1:
                        break
                    else:
                        print("❌ 유효하지 않은 언어 코드입니다. 다시 입력해주세요.")

                result_path = translator.translate_image(image_path, target_lang)
                
                if result_path:
                    if input("\n결과 이미지를 표시하시겠습니까? (y/n) > ").strip().lower() == 'y':
                        try:
                            result_image = cv2.imread(result_path)
                            cv2.imshow('예술적 번역 결과', result_image)
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()
                        except Exception as e:
                            print(f"❌ 이미지를 표시할 수 없습니다: {e}")
            
            elif choice == "2":
                print("👋 프로그램을 종료합니다.")
                break
            else:
                print("잘못된 옵션입니다. 1 또는 2를 선택해주세요.")
    
    except Exception as e:
        print(f"❌ 시작 중 심각한 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    main()



🎨 예술적 Azure 이미지 번역기
💎 아름다운 박물관 스타일 오버레이를 생성합니다.
✅ 전문가급 이미지 번역기가 성공적으로 초기화되었습니다.

1. 이미지 번역하기
2. 종료

📖 번역 가능한 언어 코드 예시:
  - ko: 한국어
  - en: 영어
  - ja: 일본어
  - zh-Hans: 중국어(간체)
  - fr: 프랑스어
  - de: 독일어
  - es: 스페인어

🎨 예술적 이미지 번역을 시작합니다: 111.png
👁️ 이미지를 분석하여 텍스트 위치를 파악하는 중...
📦 7개의 개별 텍스트 라인을 블록으로 정리했습니다.
🔄 텍스트 블록을 번역 중입니다...
🎨 세련된 예술적 오버레이를 생성 중입니다...
  ✨ 7개의 라인을 성공적으로 렌더링했습니다.
💎 예술적 결과물이 111_artistic_translated.png에 저장되었습니다.
✨ 번역 완료! 결과: 111_artistic_translated.png

1. 이미지 번역하기
2. 종료

📖 번역 가능한 언어 코드 예시:
  - ko: 한국어
  - en: 영어
  - ja: 일본어
  - zh-Hans: 중국어(간체)
  - fr: 프랑스어
  - de: 독일어
  - es: 스페인어

🎨 예술적 이미지 번역을 시작합니다: menu.jpg
👁️ 이미지를 분석하여 텍스트 위치를 파악하는 중...
📦 27개의 개별 텍스트 라인을 블록으로 정리했습니다.
🔄 텍스트 블록을 번역 중입니다...
🎨 세련된 예술적 오버레이를 생성 중입니다...
  ✨ 27개의 라인을 성공적으로 렌더링했습니다.
💎 예술적 결과물이 menu_artistic_translated.png에 저장되었습니다.
✨ 번역 완료! 결과: menu_artistic_translated.png

1. 이미지 번역하기
2. 종료

📖 번역 가능한 언어 코드 예시:
  - ko: 한국어
  - en: 영어
  - ja: 일본어
  - zh-Hans: 중국어(간체)
  - fr: 프랑스어
  - de: 독일어
  - es: 스페인어

🎨 예